# Applied Data Science Capstone Course by Coursera

#### Week 3 Part 1

Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.


##### 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


##### 2. Scrap data from Wikipedia page into a DataFrame


In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')


In [4]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []


In [16]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell


In [17]:
# create a new DataFrame from the three lists
import pandas as pd
toronto_df =pd.DataFrame(postalCodeList,columns=['PostalCode'])
toronto_df['Borough']= boroughList
toronto_df['Neighborhood']=neighborhoodList
toronto_df.reset_index(drop=True, inplace=True)
toronto_df = toronto_df.replace('\n',' ', regex=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### 3. Drop cells with a borough that is "Not assigned"

In [18]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[~toronto_df.Borough.str.contains("Not assigned")]
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### 4. Group neighborhoods in the same borough

In [19]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


#### 5. For Neighborhood="Not assigned", make the value the same as Borough


In [20]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


#### 7. Finally, print the number of rows of the cleaned dataframe


In [21]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape


(103, 3)

#### 8. Load the coordinates from the csv file on Coursera


In [22]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 9. Merge two tables to get the coordinates

In [32]:
# merge two table on the column "PostalCode"
result = pd.concat([toronto_df_grouped,coordinates],axis = 1 , join ="inner")
result.head()

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [33]:
#remove the duplicated column
toronto_df_new = result.loc[:,~result.columns.duplicated()]
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### complected